## Pauli Transfer Matrix
The Pauli Transfer Matrix(PTM) of a (potentially unitary) matrix is its representation in Pauli basis, i.e., how it acts on each Pauli string. While this package is based on transforming Pauli strings into one or many Pauli strings, most gates are not defined via the actual PT matrix. However, there are some tools that you can use to work with matrices, both in 0/1 basis and in Pauli basis, potentially to define your own gates. 

In [1]:
using PauliPropagation
using LinearAlgebra

Let us generate a random 1-qubit unitary matrix via Pauli matrix exponentials:

In [2]:
# The Pauli matrices are not exported
using PauliPropagation: Xmat, Ymat, Zmat 

U = exp(-im * (randn() * Xmat + randn() * Ymat + randn() * Zmat))

2×2 Matrix{ComplexF64}:
 -0.615763+0.758573im   -0.212069-0.0207547im
  0.212069-0.0207547im  -0.615763-0.758573im

Verify that $U$ is unitary via $U \cdot U^\dagger = U^\dagger \cdot U = \mathbb{1}$,

In [3]:
U * U' ≈ U' * U ≈ I(2)

true

This unitary is in the very common 0/1 basis, also called the computational basis.
Here is how you can transform it into the Pauli basis:

In [4]:
# note the default `heisenberg=true` kwarg
Uptm = calculateptm(U)

4×4 transpose(::Matrix{Float64}) with eltype Float64:
 1.0   0.0        0.0        0.0
 0.0  -0.240811   0.943004   0.22968
 0.0  -0.925399  -0.151726  -0.347299
 0.0  -0.292656  -0.296179   0.909192

This by default returns the PTM of `U` in the **Heisenberg** picture, i.e., how it acts in the backpropagation of Pauli strings - the default in this package.
To get the Schrödinger version, you can take the transpose of this matrix or call `calculateptm(U, heisenberg=false)`.

To convince ourselves that `Uptm` is also a unitary in this basis, check $U_{ptm} \cdot U_{ptm}^T = U_{ptm}^T \cdot U_{ptm} = \mathbb{1}$ due to unitaries being real-valued in this basis.

In [5]:
Uptm * transpose(Uptm) ≈ transpose(Uptm) * Uptm ≈ I(4)

true

Great, but what does this unitary even represent? We mentioned that it represents the action of `U` on Pauli strings. A 1-qubit gate can act on 4 Paulis, `I`, `X`, `Y`, and `Z`, each being represented as $(1, 0, 0, 0)^T$, $(0, 1, 0, 0)^T$, $(0, 0, 1, 0)^T$, and $(0, 0, 0, 1)^T$, respectively. `Uptm` thus describes how each of those column vectors or an arbitrary sum thereof is transformed.

You might already find use for this, but we also support transforming these PTMs into PT maps that can be turned into gates.

In [6]:
ptmap = totransfermap(Uptm)

4-element Vector{Vector{Tuple{UInt8, Float64}}}:
 [(0x00, 1.0)]
 [(0x01, -0.24081087775135707), (0x02, -0.9253986065371159), (0x03, -0.29265600997721686)]
 [(0x01, 0.9430043382394943), (0x02, -0.15172611582501455), (0x03, -0.296179006410224)]
 [(0x01, 0.22968008015087893), (0x02, -0.34729901352554715), (0x03, 0.909192199694909)]

Remember that we encode our Pauli strings in integers, with single-qubit Paulis being 0 (`I`), 1 (`X`), 2 (`Y`), 3 (`Z`). If you index into `ptmap` with those numbers + 1, you will get the corresponding output Paulis with their coefficients. In other words, each entry of a PT map corresponds to a column of the PTM. The Paulis will be set onto the qubit index and the coefficients will be multiplied to the incoming Pauli string's coefficient.

What we also see is that this unitary is 3-branching in Pauli basis. `X`, `Y`, and `Z` Paulis will each map to all three with different coefficients. We can define a `TransferMapGate` from PT map, specifying on what qubit it acts (here qubit 1).

In [7]:
g = TransferMapGate(ptmap, 1)

TransferMapGate{UInt8, Float64}(Vector{Tuple{UInt8, Float64}}[[(0x00, 1.0)], [(0x01, -0.24081087775135707), (0x02, -0.9253986065371159), (0x03, -0.29265600997721686)], [(0x01, 0.9430043382394943), (0x02, -0.15172611582501455), (0x03, -0.296179006410224)], [(0x01, 0.22968008015087893), (0x02, -0.34729901352554715), (0x03, 0.909192199694909)]], [1])

Keep in mind, however, that this gate is not parametrized. It always acts the same. 

In [8]:
g isa StaticGate

true

Finally, let us define a circuit consisting of this gate on every qubit: 

In [9]:
# 6 qubits
nq = 6

# define the circuit as a vector of gates
circuit = [TransferMapGate(ptmap, qind) for qind in 1:nq];

# make the observable a random global Pauli string because the gate acts trivially on identities `I`
pstr = PauliString(nq, [rand((:X, :Y, :Z)) for _ in 1:nq], 1:nq)

PauliString(nqubits: 6, 1.0 * YYXZXZ)

In [10]:
psum = propagate(circuit, pstr)

PauliSum(nqubits: 6, 729 Pauli terms:
 0.019079 * ZXYYYX
 -0.0023742 * XZZYXY
 -0.00020787 * YZXYXX
 -0.0080917 * XYZXYZ
 0.001291 * ZZXZZX
 0.001292 * XZXYXX
 0.00064202 * YZZXYX
 7.0424e-5 * YYXXXX
 0.00046423 * YZZYZY
 0.00033879 * YYXXZZ
 0.0012533 * ZZZXYX
 -0.019211 * XXYYZX
 -0.0066582 * XYYXXZ
 -0.00020787 * YZXXXY
 -0.0012153 * YZZYZZ
 0.0020301 * YZYXYX
 -0.00085441 * XZXXXX
 -0.0038429 * ZYYYZZ
 -0.0023724 * ZZZZZY
 -0.0019536 * ZXXYXY
  ⋮)

And there you go, you can now easily define gates from their matrix representations, both in 0/1 basis or Pauli basis.

### Parametrized PTMs

Remember when we use `TransferMapGate`, we need a fixed matrix representation which is then transformed into the PTM. We can also compute that matrix for parametrized gates if `tomatrix()` is defined for them.
Here is how to do that for a `PauliRotation`:

In [11]:
U = tomatrix(PauliRotation(:X, 1), π/4)
ptm = calculateptm(U)

4×4 transpose(::Matrix{Float64}) with eltype Float64:
 1.0  0.0   0.0       0.0
 0.0  1.0   0.0       0.0
 0.0  0.0   0.707107  0.707107
 0.0  0.0  -0.707107  0.707107

Then `TransferMapGate` will treat the ptm as a static gate, because it has a fixed parameter.

In [12]:
Uptm = totransfermap(ptm)
TransferMapGate(Uptm, 1)

TransferMapGate{UInt8, Float64}(Vector{Tuple{UInt8, Float64}}[[(0x00, 1.0)], [(0x01, 1.0)], [(0x02, 0.7071067811865474), (0x03, -0.7071067811865474)], [(0x02, 0.7071067811865474), (0x03, 0.7071067811865474)]], [1])

### Compressing Circuits into one PT map

One functionality which may make some gate sequences more performant to simulate is the ability to compress them into one PT map via `totransfermap()`.
Parameters for parametrized gates can be passed as well.

In [13]:
nq = 5

circuit = Gate[]
append!(circuit, CliffordGate(:CNOT, pair) for pair in bricklayertopology(nq))
append!(circuit, CliffordGate(:H, ii) for ii in 1:nq)
append!(circuit, PauliRotation(:Y, ii) for ii in 1:nq)

# the angles for the Pauli rotations
thetas = [π / 8 for _ in 1:nq]

# compile everything into one
ptmap = totransfermap(nq, circuit, thetas)
circuit_map = TransferMapGate(ptmap, 1:nq)

## feel free to print this huge PT map
# print("Now that is a mess! But it is correct:\n ", ptmap)

TransferMapGate{UInt16, Float64}(Vector{Tuple{UInt16, Float64}}[[(0x0000, 1.0)], [(0x0005, 0.3826834323650898), (0x0003, 0.9238795325112867)], [(0x0006, -1.0)], [(0x0005, 0.9238795325112867), (0x0003, -0.3826834323650898)], [(0x0054, 0.3826834323650898), (0x000f, 0.9238795325112867)], [(0x0051, 0.14644660940672624), (0x000a, -0.3535533905932738), (0x000c, 0.8535533905932737), (0x0057, 0.3535533905932738)], [(0x0009, -0.9238795325112867), (0x0052, -0.3826834323650898)], [(0x0051, 0.3535533905932738), (0x000a, -0.8535533905932737), (0x000c, -0.3535533905932738), (0x0057, -0.14644660940672624)], [(0x005b, -1.0)], [(0x005e, -0.3826834323650898), (0x0058, -0.9238795325112867)]  …  [(0x0302, -0.051776695296636886), (0x019d, -0.051776695296636886), (0x01a9, -0.30177669529663687), (0x01c6, -0.125), (0x002d, 0.125), (0x0076, 0.30177669529663687), (0x0359, -0.125), (0x0286, 0.125), (0x02b2, -0.125), (0x02e9, -0.30177669529663687), (0x0336, 0.051776695296636886), (0x0019, -0.7285533905932737), (0

In [14]:
# it can be apply in one go
pstr = PauliString(nq, [:X for _ in 1:nq], 1:nq)
psum1 = propagate(circuit_map, pstr)

PauliSum(nqubits: 5, 32 Pauli terms:
 0.11548 * ZXIIZ
 -0.11548 * YZYXZ
 -0.047835 * ZXZYY
 -0.019814 * XIZYY
 -0.11548 * YYXXZ
 0.047835 * IZXII
 -0.047835 * YZXZY
 0.019814 * XIZZX
 0.27881 * IZXXZ
 0.047835 * XXZXI
 0.047835 * YYYYX
 -0.11548 * IZYZY
 0.27881 * ZIIZX
 0.047835 * YYYZY
 0.11548 * ZIZXI
 0.047835 * ZXZZX
 -0.11548 * IYXYX
 0.11548 * XXIZX
 -0.019814 * YZYII
 0.019814 * ZXIXI
  ⋮)

Let's verify that we get the same answer using a circuit constructed from `TransferMapGate`s and natively supported gates.

In [15]:
psum2 = propagate(circuit, pstr, thetas)
@show psum1 == psum2

psum1 == psum2 = true


true

Finally, lets's compare their performance

In [16]:
using BenchmarkTools
println("Compiled circuit via transfer map:")
@btime propagate($circuit_map, $pstr);
println("Original circuit:")
@btime propagate($circuit, $pstr, $thetas);

Compiled circuit via transfer map:
  1.877 μs (61 allocations: 4.55 KiB)
Original circuit:
  8.852 μs (85 allocations: 5.84 KiB)


There are quite a few nuances to discuss here. First, the size of the circuit in terms of the number of qubits $n$ that can be compressed is limited. This is due to the $4^n$ best-case and $8^n$ worst-case memory and time scaling (depending on the branching behavior). It may still be possible and beneficial to compress common gate sequences, for example few-qubit entangling blocks occuring often throughout the circuit. However, it is not guaranteed that this type of PT map compression yields faster gates, especially when compressing few and otherwise highly optimized gates. The `TransferMapGate` is generic, but its application is not type stable and will induce a lot of memory movement. That being said - try it out for your use-case!